In [3]:
#########################################################################
# LIBRERIAS NECESARIAS
#########################################################################
import gymnasium as gym
import numpy as np
import sinergym
from sinergym.utils.wrappers import LoggerWrapper
import torch

import tensorflow as tf
from stable_baselines3 import PPO
from tensorflow.keras.layers import concatenate
import keras
import matplotlib.pyplot as plt
from keras import layers
import copy
from sklearn.model_selection import train_test_split
import csv

import os
from stable_baselines3.common.vec_env import DummyVecEnv

2023-07-21 06:59:59.319910: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-21 06:59:59.505904: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-21 07:00:00.229187: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
############################################################################
# Variables globales
###########################################################################
BATCH_SIZE = 28032
EPOCHS =100
EPISODES = 1

# 5ZONE

In [5]:
# Definimos el entorno
env = gym.make('Eplus-5Zone-hot-discrete-v1')
env = LoggerWrapper(env)

# Obtenemos el espacio de estados del entorno
ob_space = env.observation_space
ac_space = env.action_space


[2023-07-21 07:00:02,098] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-07-21 07:00:02,099] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-07-21 07:00:02,100] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating building model OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-07-21 07:00:02,100] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2023-07-21 07:00:02,101] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Setting up action definition in building model if exists...


In [6]:
model = PPO(policy="MlpPolicy", env=env, verbose=0)

In [7]:
model.learn(total_timesteps=25)

[2023-07-21 07:00:02,140] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 07:00:02,221] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res24/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


In [8]:
reward = 0
success_num = 0

# Por cada episodio
for iteration in range(EPOCHS):
    # Inicializo todas las variables
    observations = []
    actions = []
    rewards = []
    run_policy_steps = 0

    truncated = False
    terminated = False

    # La primera acción de cada episodio se crea con la red neuronal

    obs, _ = env.reset()


    # --Actualización de variables: ojo no introduzco el estado y accion inicial, solo introduzco los de PPO
    observations.append(obs)  # S_0

   
    # Por cada steps en cada episodio, mientras no se llegue a un
    # estado terminal o un estado malo
    while terminated != True and truncated != True:
        # --Aumentar el numero de steps
        run_policy_steps += 1

        action, _ = model.predict(obs)

        action = int(action)

        # --Muevo al Agente al siguiente estado
        next_obs, reward, terminated, truncated, info = env.step(action)

        # --Actualización de variables
        actions.append(action)  # A_i-1
        rewards.append(reward)  # R_i-1

        # --Si llegamos a un estado final, el juego ha finalizado!!!
        # --Se configura el tablero de nuevo
        if terminated:
            obs = env.reset()
            reward = -1
            break
        else:
            observations.append(next_obs)  # O_i
            obs = next_obs

    # Ver si el episodio ha obtendo una recompensa total igual o
    # superior a 195
    if sum(rewards) >= 195:
        success_num += 1
        if success_num >= 100:
            break
    else:
        success_num = 0

observations = np.reshape(observations,newshape=[-1] + list(ob_space.shape))

actions = np.array(actions).astype(dtype=np.int32)


[2023-07-21 07:00:08,350] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 07:00:08,351] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 07:00:08,432] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res24/Eplus-env-sub_run2
[2023-07-21 07:00:22,890] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 07:00:22,891] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 07:00:22,972] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res24/Eplus-env-sub_run3


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[2023-07-21 07:00:26,958] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 07:00:26,960] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 07:00:27,044] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res24/Eplus-env-sub_run4
[2023-07-21 07:00:40,746] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 07:00:40,747] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 07:00:40,827] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res24/Eplus-env-sub_run5
[2023-07-21 07:00:44,644] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully

**A continuación, probamos como actuaría una agente con la política obtenida con PPO**

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


[2023-07-21 07:29:43,617] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 07:29:43,618] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 07:29:43,720] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res24/Eplus-env-sub_run202
[2023-07-21 07:30:00,467] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 07:30:00,469] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 07:30:00,549] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res24/Eplus-env-sub_run203
[2023-07-21 07:30:16,698] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successf

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=5)

In [10]:

print(f"Recompensa promedio: {mean_reward} +/- {std_reward}")


Recompensa promedio: -42662.03333095694 +/- 0.0


# DATACENTER

# Definimos el entorno
env = gym.make('Eplus-Datacenter-hot-discrete-v1')
env = LoggerWrapper(env)

# Obtenemos el espacio de estados del entorno
ob_space = env.observation_space
ac_space = env.action_space


In [11]:
model = PPO(policy="MlpPolicy", env=env, verbose=0)

In [12]:
model.learn(total_timesteps=25)

[2023-07-21 07:31:34,662] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 07:31:34,663] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 07:31:34,763] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res24/Eplus-env-sub_run208


In [13]:
reward = 0
success_num = 0

# Por cada episodio
for iteration in range(EPOCHS):
    # Inicializo todas las variables
    observations = []
    actions = []
    rewards = []
    run_policy_steps = 0

    truncated = False
    terminated = False

    # La primera acción de cada episodio se crea con la red neuronal

    obs, _ = env.reset()


    # --Actualización de variables: ojo no introduzco el estado y accion inicial, solo introduzco los de PPO
    observations.append(obs)  # S_0

   
    # Por cada steps en cada episodio, mientras no se llegue a un
    # estado terminal o un estado malo
    while terminated != True and truncated != True:
        # --Aumentar el numero de steps
        run_policy_steps += 1

        action, _ = model.predict(obs)

        action = int(action)

        # --Muevo al Agente al siguiente estado
        next_obs, reward, terminated, truncated, info = env.step(action)

        # --Actualización de variables
        actions.append(action)  # A_i-1
        rewards.append(reward)  # R_i-1

        # --Si llegamos a un estado final, el juego ha finalizado!!!
        # --Se configura el tablero de nuevo
        if terminated:
            obs = env.reset()
            reward = -1
            break
        else:
            observations.append(next_obs)  # O_i
            obs = next_obs

    # Ver si el episodio ha obtendo una recompensa total igual o
    # superior a 195
    if sum(rewards) >= 195:
        success_num += 1
        if success_num >= 100:
            break
    else:
        success_num = 0

observations = np.reshape(observations,newshape=[-1] + list(ob_space.shape))

actions = np.array(actions).astype(dtype=np.int32)


[2023-07-21 07:31:41,596] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 07:31:41,598] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 07:31:41,697] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res24/Eplus-env-sub_run209
[2023-07-21 07:32:03,420] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 07:32:03,421] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 07:32:03,525] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res24/Eplus-env-sub_run210
[2023-07-21 07:32:07,882] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successf

In [14]:
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=5)

[2023-07-21 08:12:55,094] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 08:12:55,096] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 08:12:55,179] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res24/Eplus-env-sub_run409
[2023-07-21 08:13:10,651] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 08:13:10,653] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 08:13:10,740] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res24/Eplus-env-sub_run410
[2023-07-21 08:13:26,251] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successf

In [15]:
print(f"Recompensa promedio: {mean_reward} +/- {std_reward}")

Recompensa promedio: -48925.4385437468 +/- 0.0


# WAREHOUSE

In [17]:
# Definimos el entorno
env = gym.make('Eplus-warehouse-hot-discrete-v1')
env = LoggerWrapper(env)

# Obtenemos el espacio de estados del entorno
ob_space = env.observation_space
ac_space = env.action_space


[2023-07-21 08:17:10,897] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-07-21 08:17:10,898] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-07-21 08:17:10,900] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:Updating building model OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-07-21 08:17:10,900] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2023-07-21 08:17:10,901] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:Setting up action definition in building model if exists...


In [18]:
model = PPO(policy="MlpPolicy", env=env, verbose=0)

In [19]:
model.learn(total_timesteps=25)

[2023-07-21 08:17:14,468] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 08:17:14,577] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-warehouse-hot-discrete-v1-res18/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


In [20]:
reward = 0
success_num = 0

# Por cada episodio
for iteration in range(EPOCHS):
    # Inicializo todas las variables
    observations = []
    actions = []
    rewards = []
    run_policy_steps = 0

    truncated = False
    terminated = False

    # La primera acción de cada episodio se crea con la red neuronal

    obs, _ = env.reset()


    # --Actualización de variables: ojo no introduzco el estado y accion inicial, solo introduzco los de PPO
    observations.append(obs)  # S_0

   
    # Por cada steps en cada episodio, mientras no se llegue a un
    # estado terminal o un estado malo
    while terminated != True and truncated != True:
        # --Aumentar el numero de steps
        run_policy_steps += 1

        action, _ = model.predict(obs)

        action = int(action)

        # --Muevo al Agente al siguiente estado
        next_obs, reward, terminated, truncated, info = env.step(action)

        # --Actualización de variables
        actions.append(action)  # A_i-1
        rewards.append(reward)  # R_i-1

        # --Si llegamos a un estado final, el juego ha finalizado!!!
        # --Se configura el tablero de nuevo
        if terminated:
            obs = env.reset()
            reward = -1
            break
        else:
            observations.append(next_obs)  # O_i
            obs = next_obs

    # Ver si el episodio ha obtendo una recompensa total igual o
    # superior a 195
    if sum(rewards) >= 195:
        success_num += 1
        if success_num >= 100:
            break
    else:
        success_num = 0

observations = np.reshape(observations,newshape=[-1] + list(ob_space.shape))

actions = np.array(actions).astype(dtype=np.int32)

[2023-07-21 08:17:31,295] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 08:17:31,296] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 08:17:31,422] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-warehouse-hot-discrete-v1-res18/Eplus-env-sub_run2
[2023-07-21 08:18:13,321] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 08:18:13,323] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 08:18:13,430] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-warehouse-hot-discrete-v1-res18/Eplus-env-sub_run3


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[2023-07-21 08:18:26,733] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 08:18:26,734] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 08:18:26,824] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-warehouse-hot-discrete-v1-res18/Eplus-env-sub_run4
[2023-07-21 08:19:03,309] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 08:19:03,312] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 08:19:03,412] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-warehouse-hot-discrete-v1-res18/Eplus-env-sub_run5
[2023-07-21 08:19:15,070] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:Energ

In [21]:
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=5)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


[2023-07-21 09:42:07,256] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 09:42:07,258] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 09:42:07,352] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-warehouse-hot-discrete-v1-res18/Eplus-env-sub_run202
[2023-07-21 09:42:33,426] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-21 09:42:33,428] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-21 09:42:33,517] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-warehouse-hot-discrete-v1-res18/Eplus-env-sub_run203
[2023-07-21 09:43:00,376] EPLUS_ENV_warehouse-hot-discrete-v1_MainThread_ROOT INFO:E

In [22]:
print(f"Recompensa promedio: {mean_reward} +/- {std_reward}")

Recompensa promedio: -44725.37481083907 +/- 0.0
